In this notebook we are performing XGBoost training with GPU acceleration

In [ ]:
# == Libraries ==
import os
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from optuna.integration import XGBoostPruningCallback
from datetime import timedelta
import time



We will load all 10 numpy arrays that correspond to our 10 batches and contain Connolly points as feature vectors (32 features & 1 label). Then we will merge them again. We better handled them before in the PDB to Connolly points process in batches but for the training it is better to merge.

In [ ]:

# ===  Load and split all batches (Train/Test only) ===
INPUT_FOLDER = "input_numpy"
N_BATCHES = 10

X_train_all, y_train_all = [], []
X_test_all, y_test_all = [], []

for i in range(1, N_BATCHES + 1):
    batch_path = os.path.join(INPUT_FOLDER, f"batch_{i}.npy")
    data = np.load(batch_path)
    X = data[:, :-1]
    y = data[:, -1]

    # Stratified split: 70% train, 30% test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    X_train_all.append(X_train)
    y_train_all.append(y_train)
    X_test_all.append(X_test)
    y_test_all.append(y_test)

# Merge all batches
X_train_all = np.concatenate(X_train_all)
y_train_all = np.concatenate(y_train_all)
X_test_all = np.concatenate(X_test_all)
y_test_all = np.concatenate(y_test_all)




Get to kno our dataset and what to expect, spoiler alert! Great class imbalance.

In [12]:
# === Optional: Print dataset summary ===
total = len(y_train_all) + len(y_test_all)
print("Dataset Summary")
print(f"Total samples: {total:,}")
print(f"Train set: {len(y_train_all):,} samples | Positives: {(y_train_all == 1).sum():,} ({(y_train_all == 1).mean()*100:.2f}%)")
print(f"Test set:  {len(y_test_all):,} samples | Positives: {(y_test_all == 1).sum():,} ({(y_test_all == 1).mean()*100:.2f}%)")


Dataset Summary
Total samples: 11,205,983
Train set: 7,844,183 samples | Positives: 426,817 (5.44%)
Test set:  3,361,800 samples | Positives: 182,921 (5.44%)


Set the function for hyperparameter tuning by selecting a wide range of hyper parameters including scale_pos_weight that has to be adjusted in order to account for class imbalance. Optuna (bayesian) is a very good method compatible with XGBoost that is used for tuning.

In [ ]:
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import numpy as np

# Use full dataset with 4 folds
X_sampled = X_train_all
y_sampled = y_train_all

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'eval_metric': 'auc',
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'gamma': trial.suggest_float('gamma', 0, 2),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 4),
        'reg_lambda': trial.suggest_float('lambda', 0, 2),
        'reg_alpha': trial.suggest_float('alpha', 0, 2),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 5.0, 25.0),
    }

    kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    aucs = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_sampled, y_sampled)):
        X_train_fold, X_val_fold = X_sampled[train_idx], X_sampled[val_idx]
        y_train_fold, y_val_fold = y_sampled[train_idx], y_sampled[val_idx]

        model = xgb.XGBClassifier(**params, n_jobs=2, verbosity=0)

        try:
            model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y_val_fold)],
                verbose=False
            )
            preds = model.predict_proba(X_val_fold)[:, 1]
            auc = roc_auc_score(y_val_fold, preds)
            aucs.append(auc)
        except Exception as e:
            print(f"Trial failed on fold {fold}: {e}")
            return 0.0
        finally:
            del model
            gc.collect()

    trial.set_user_attr("fold_aucs", aucs)
    return np.mean(aucs)




Apply the tuning and set a big number of trials (n=100). This took around 4.30 hours with GPU acceleration

In [21]:
# === Step 3 ===

study = optuna.create_study(direction="maximize")
start_time = time.time()

N_TRIALS = 100

for i in range(N_TRIALS):
    trial_start = time.time()
    study.optimize(objective, n_trials=1, show_progress_bar=False)
    trial_time = time.time() - trial_start

    total_elapsed = time.time() - start_time
    avg_per_trial = total_elapsed / (i + 1)
    remaining = N_TRIALS - (i + 1)
    eta = timedelta(seconds=int(avg_per_trial * remaining))

    print(f"Trial {i+1}/{N_TRIALS} completed in {trial_time:.1f}s | ETA: {eta}")


[I 2025-04-13 18:57:43,542] A new study created in memory with name: no-name-5d54692e-ba66-40c6-b54a-a7fb97ec31b8
[I 2025-04-13 18:59:31,139] Trial 0 finished with value: 0.7577805634556514 and parameters: {'max_depth': 5, 'learning_rate': 0.26739867211050083, 'n_estimators': 101, 'subsample': 0.6034221944535948, 'colsample_bytree': 0.7679112684037179, 'gamma': 1.4652810794505209, 'min_child_weight': 3, 'lambda': 0.8402929077808412, 'alpha': 0.8329132528257754, 'scale_pos_weight': 18.028607176160538}. Best is trial 0 with value: 0.7577805634556514.


Trial 1/100 completed in 107.6s | ETA: 2:57:32


[I 2025-04-13 19:01:49,591] Trial 1 finished with value: 0.7580496255465687 and parameters: {'max_depth': 4, 'learning_rate': 0.2259277700061942, 'n_estimators': 258, 'subsample': 0.8430514535088888, 'colsample_bytree': 0.647170522855383, 'gamma': 0.015780903874117902, 'min_child_weight': 2, 'lambda': 0.5808646867692167, 'alpha': 0.3962820538018581, 'scale_pos_weight': 24.97534184332912}. Best is trial 1 with value: 0.7580496255465687.


Trial 2/100 completed in 138.5s | ETA: 3:20:56


[I 2025-04-13 19:03:53,286] Trial 2 finished with value: 0.7395841939090649 and parameters: {'max_depth': 3, 'learning_rate': 0.2759379867116163, 'n_estimators': 220, 'subsample': 0.7683125489698739, 'colsample_bytree': 0.6332557562014727, 'gamma': 0.764835710198654, 'min_child_weight': 1, 'lambda': 1.6179283911976279, 'alpha': 1.1087587339217593, 'scale_pos_weight': 9.984690532291332}. Best is trial 1 with value: 0.7580496255465687.


...

Trial 98/100 completed in 108.5s | ETA: 0:04:57


[I 2025-04-13 23:03:13,210] Trial 98 finished with value: 0.7479141079058109 and parameters: {'max_depth': 3, 'learning_rate': 0.2949071212721645, 'n_estimators': 296, 'subsample': 0.8509038992724206, 'colsample_bytree': 0.8468711110808823, 'gamma': 1.8463274933650828, 'min_child_weight': 1, 'lambda': 1.436138390646801, 'alpha': 0.40798285407546336, 'scale_pos_weight': 15.347046385812476}. Best is trial 94 with value: 0.8116306670007959.


Trial 99/100 completed in 136.9s | ETA: 0:02:28


[I 2025-04-13 23:05:47,345] Trial 99 finished with value: 0.8068923487320633 and parameters: {'max_depth': 6, 'learning_rate': 0.27959441110810246, 'n_estimators': 273, 'subsample': 0.8612564508454825, 'colsample_bytree': 0.8092266846591465, 'gamma': 1.6774589791456678, 'min_child_weight': 1, 'lambda': 1.7479517694697881, 'alpha': 0.6508054047816392, 'scale_pos_weight': 10.589475251499003}. Best is trial 94 with value: 0.8116306670007959.


Trial 100/100 completed in 154.1s | ETA: 0:00:00


Trial 94/100 completed in 157.7s | ETA: 0:14:53
[I 2025-04-13 22:53:46,232] Trial 94 finished with value: 0.8116306670007959 and parameters: {'max_depth': 6, 'learning_rate': 0.2948641667274368, 'n_estimators': 300, 'subsample': 0.871795472197383, 'colsample_bytree': 0.818803808976565, 'gamma': 1.883267048382789, 'min_child_weight': 1, 'lambda': 1.5493470554947497, 'alpha': 0.45298380003513483, 'scale_pos_weight': 13.136280206298453}. Best is trial 94 with value: 0.8116306670007959. 
These are the hyperparameters of the best trial.

In [ ]:
# === Train final model on entire training set with best params ===
best_params = study.best_params.copy()
best_params.update({
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda'  
})

final_model = xgb.XGBClassifier(**best_params)

# Train on all training data (no validation split, no early stopping)
final_model.fit(X_train_all, y_train_all, verbose=True)

# === Evaluate on test set ===
from sklearn.metrics import roc_auc_score

test_preds = final_model.predict_proba(X_test_all)[:, 1]
test_auc = roc_auc_score(y_test_all, test_preds)
print(f"Final Test AUC: {test_auc:.4f}")


Final Test AUC: 0.8133


Not the best but we know this is quite a difficult problem and we did a very extensive tuning. Let's evaluate all the other metrics now to check how we did.

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Convert probabilities to binary predictions
test_labels = (test_preds >= 0.5).astype(int)

# Compute standard metrics
test_accuracy = accuracy_score(y_test_all, test_labels)
test_precision = precision_score(y_test_all, test_labels, zero_division=0)
test_recall = recall_score(y_test_all, test_labels, zero_division=0)
test_f1 = f1_score(y_test_all, test_labels, zero_division=0)
test_conf_matrix = confusion_matrix(y_test_all, test_labels)
test_report = classification_report(y_test_all, test_labels, digits=4)

print("\nFinal Test Metrics:")
print(f"AUC:       {test_auc:.4f}")
print(f"Accuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print("\nConfusion Matrix:\n", test_conf_matrix)
print("\nFull Classification Report:\n", test_report)



✅ Final Test Metrics:
AUC:       0.8133
Accuracy:  0.8309
Precision: 0.1819
Recall:    0.6025
F1 Score:  0.2795

Confusion Matrix:
 [[2683276  495603]
 [  72714  110207]]

Full Classification Report:
               precision    recall  f1-score   support

         0.0     0.9736    0.8441    0.9042   3178879
         1.0     0.1819    0.6025    0.2795    182921

    accuracy                         0.8309   3361800
   macro avg     0.5778    0.7233    0.5918   3361800
weighted avg     0.9305    0.8309    0.8702   3361800



Let's extract feature importance

In [ ]:
import pandas as pd
import numpy as np

# === Show feature importances ===
importances = final_model.feature_importances_
importance_df = pd.DataFrame({
    "Feature Index": np.arange(len(importances)),
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

display(importance_df)



,Feature Index,Importance
15,15,0.121020
1,1,0.066357
20,20,0.054123
24,24,0.052567
31,31,0.050008
30,30,0.046646
29,29,0.043578
25,25,0.039008
11,11,0.031604
3,3,0.029356


It looks like the best feature here is when a Connolly point is both a Hydrogen Donor and Acceptor at the same time!

Save the model into different formats

In [27]:
from joblib import dump
import time

# Create a unique timestamped name
timestamp = time.strftime("%Y%m%d_%H%M%S")
model_name = f"xgboost_binding_site_model_{timestamp}"

# Save as native XGBoost binary
final_model.save_model(f"{model_name}.json")  # Good for reloading with XGBoost directly

# Save as Python pickle (Joblib format) — great for reuse in any Python project
dump(final_model, f"{model_name}.pkl")

# Save feature importances too (optional)
importance_df.to_csv(f"{model_name}_feature_importances.csv", index=False)

print(f"Model saved as {model_name}.json and {model_name}.pkl")


Model saved as xgboost_binding_site_model_20250414_115052.json and xgboost_binding_site_model_20250414_115052.pkl
